In [13]:
import gzip
from collections import defaultdict

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
data = []
count = 0
for d in readGz('ratebeer.json.gz'):
    count += 1
    data.append(d)
    if (count >= 100000):
        break

In [14]:
itemPerUser = defaultdict(list)
userPerItem = defaultdict(list)
brewerSet = set()
for d in data:
    itemPerUser[d['review/profileName']].append((d['beer/beerId'], d['review/overall']))
    userPerItem[d['beer/beerId']].append((d['review/profileName'], d['review/overall']))
    brewerSet.add(d['beer/brewerId'])